In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1695305285268_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-2>

In [2]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# creating custom schema

schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("weekday", StringType(), True),
    StructField("hour", IntegerType(), True),
    StructField("atm_status", StringType(), True),
    StructField("atm_id", StringType(), True),
    StructField("atm_manufacturer", StringType(), True),
    StructField("atm_location", StringType(), True),
    StructField("atm_streetname", StringType(), True),
    StructField("atm_street_number", IntegerType(), True),
    StructField("atm_zipcode", IntegerType(), True),
    StructField("atm_lat", DoubleType(), True),
    StructField("atm_lon", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("card_type", StringType(), True),
    StructField("transaction_amount", IntegerType(), True),
    StructField("service", StringType(), True),
    StructField("message_code", StringType(), True),
    StructField("message_text", StringType(), True),
    StructField("weather_lat", DoubleType(), True),
    StructField("weather_lon", DoubleType(), True),
    StructField("weather_city_id", IntegerType(), True),
    StructField("weather_city_name", StringType(), True),
    StructField("temp", DoubleType(), True),
    StructField("pressure", IntegerType(), True),
    StructField("humidity", IntegerType(), True),
    StructField("wind_speed", IntegerType(), True),
    StructField("wind_deg", IntegerType(), True),
    StructField("rain_3h", DoubleType(), True),
    StructField("clouds_all", IntegerType(), True),
    StructField("weather_id", IntegerType(), True),
    StructField("weather_main", StringType(), True),
    StructField("weather_description", StringType(), True)
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# reading the data from HDFS

df = spark.read \
    .format("csv") \
    .option("header", "false") \
    .schema(schema) \
    .load("data/", sep='|')


df.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+--------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency| card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+--------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-----------+------

In [6]:
# verify count of total records 
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [7]:
df.rdd.getNumPartitions()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4

In [8]:
# check if the schema is correct
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

## Creating Location dimension

In [9]:
# select only distinct records in all dimension tables.
df_location = df.select('atm_location','atm_streetname','atm_street_number','atm_zipcode','atm_lat','atm_lon').distinct()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_location.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-----------------+-----------+-------+-------+
|    atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+----------------+-------------------+-----------------+-----------+-------+-------+
|         Kolding|           Vejlevej|              135|       6000| 55.505|  9.457|
|  Skelagervej 15|        Skelagervej|               15|       9000| 57.023|  9.891|
|Intern HolbÃƒÂ¦k|        Slotsvolden|                7|       4300| 55.718| 11.704|
|          Odense|       FÃƒÂ¦lledvej|                3|       5000| 55.394|  10.37|
|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|               12|       7430| 56.139|  9.154|
+----------------+-------------------+-----------------+-----------+-------+-------+
only showing top 5 rows

In [11]:
df_location.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [12]:
df_location.select('*').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-----------------+-----------+-------+-------+
|    atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+----------------+-------------------+-----------------+-----------+-------+-------+
|         Kolding|           Vejlevej|              135|       6000| 55.505|  9.457|
|  Skelagervej 15|        Skelagervej|               15|       9000| 57.023|  9.891|
|Intern HolbÃƒÂ¦k|        Slotsvolden|                7|       4300| 55.718| 11.704|
|          Odense|       FÃƒÂ¦lledvej|                3|       5000| 55.394|  10.37|
|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|               12|       7430| 56.139|  9.154|
+----------------+-------------------+-----------------+-----------+-------+-------+
only showing top 5 rows

In [13]:
# creating a primary key 'location_id' based on row_number

window_spec = Window.partitionBy().orderBy(df_location['atm_location'])
df_location = df_location.select(row_number().over(window_spec).alias('location_id'), '*')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_location.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+---------------+-----------------+-----------+-------+-------+
|location_id|        atm_location| atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+-----------+--------------------+---------------+-----------------+-----------+-------+-------+
|          1|             Aabybro|   ÃƒËœstergade|                6|       9440| 57.162|   9.73|
|          2|      Aalborg Hallen|   Europa Plads|                4|       9000| 57.044|  9.913|
|          3|Aalborg Storcente...|       Hobrovej|              452|       9200| 57.005|  9.876|
|          4|Aalborg Storcente...|       Hobrovej|              452|       9200| 57.005|  9.876|
|          5|         Aalborg Syd|       Hobrovej|              440|       9200| 57.005|  9.881|
|          6|           AalbÃƒÂ¦k|     Centralvej|                5|       9982| 57.593| 10.412|
|          7|              Aarhus| SÃƒÂ¸nder Alle|               11|       8000| 56.153| 10.206|
|          8|              Aar

In [15]:
df_location.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [16]:
df_location.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------------+-----------------+-----------+-------+-------+
|location_id|        atm_location|    atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+-----------+--------------------+------------------+-----------------+-----------+-------+-------+
|          1|             Aabybro|      ÃƒËœstergade|                6|       9440| 57.162|   9.73|
|          2|      Aalborg Hallen|      Europa Plads|                4|       9000| 57.044|  9.913|
|          3|Aalborg Storcente...|          Hobrovej|              452|       9200| 57.005|  9.876|
|          4|Aalborg Storcente...|          Hobrovej|              452|       9200| 57.005|  9.876|
|          5|         Aalborg Syd|          Hobrovej|              440|       9200| 57.005|  9.881|
|          6|           AalbÃƒÂ¦k|        Centralvej|                5|       9982| 57.593| 10.412|
|          7|              Aarhus|    SÃƒÂ¸nder Alle|               11|       8000| 56.153| 10.206|


In [17]:
df_location.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- location_id: integer (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)

## Creating ATM dimension

In [18]:
# Also select the columns 'atm_location','atm_streetname','atm_street_number','atm_zipcode','atm_lat','atm_lon' to join
# the table with location table using these columns to fetch the location_id

df_atm = df.select('atm_id','atm_manufacturer','atm_location','atm_streetname',
                   'atm_street_number', 'atm_zipcode','atm_lat','atm_lon').distinct()
df_atm.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [19]:
df_atm.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------+--------------------+-----------------+-----------------+-----------+-------+-------+
|atm_id|atm_manufacturer|        atm_location|   atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+------+----------------+--------------------+-----------------+-----------------+-----------+-------+-------+
|    59| Diebold Nixdorf|NykÃƒÂ¸bing Mors ...|      Kirketorvet|                1|       7900| 56.795|   8.86|
|    41| Diebold Nixdorf|              Skagen|Sct. Laurentiivej|               36|       9990| 57.723|  10.59|
|    40| Diebold Nixdorf|       Frederikshavn|     Danmarksgade|               48|       9900| 57.441| 10.537|
|    87|             NCR|Aalborg Storcente...|         Hobrovej|              452|       9200| 57.005|  9.876|
|    31|             NCR|            Slagelse|  Mariendals Alle|               29|       4200| 55.398| 11.342|
+------+----------------+--------------------+-----------------+-----------------+-----------+-------+-------+
o

In [20]:
# creating views for atm and location dimension to join them using sql and create foreign key location_id
# in atm table based on location table primary key

df_atm.createOrReplaceTempView('atm')
df_location.createOrReplaceTempView('loc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# joining on atm_location to fetch the location_id from location dimension.
# doing left join to fetch the location_id for each atm_id
# joining is possible on atm_location as atm_location

df_atm = spark.sql(" select atm.atm_id, atm.atm_manufacturer, loc.location_id \
                     from atm left join loc on \
                     atm.atm_location = loc.atm_location and \
                     atm.atm_streetname = loc.atm_streetname and \
                     atm.atm_street_number = loc.atm_street_number and \
                     atm.atm_zipcode = loc.atm_zipcode and \
                     atm.atm_lat = loc.atm_lat and \
                     atm.atm_lon = loc.atm_lon ")

df_atm.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------+-----------+
|atm_id|atm_manufacturer|location_id|
+------+----------------+-----------+
|    40| Diebold Nixdorf|         27|
|    41| Diebold Nixdorf|         82|
|    59| Diebold Nixdorf|         73|
|    97|             NCR|         51|
|    19|             NCR|         62|
|    31|             NCR|         88|
|    87|             NCR|          4|
|    74|             NCR|         61|
|    56| Diebold Nixdorf|         64|
|    51|             NCR|         29|
+------+----------------+-----------+
only showing top 10 rows

In [22]:
df_atm.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [23]:
# atm_id in fact table should be atm_number as per the atm schema
# and atm_id in atm schema should be the primary key generated using row_number

window_spec = Window.partitionBy().orderBy(df_atm['atm_id'])
df_atm = df_atm.select(row_number().over(window_spec).alias('atm_id'),
                       col('atm_id').alias('atm_number'),
                       'atm_manufacturer', 'location_id')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_atm.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+-----------+
|atm_id|atm_number|atm_manufacturer|location_id|
+------+----------+----------------+-----------+
|     1|         1|             NCR|         74|
|     2|        10|             NCR|         76|
|     3|       100|             NCR|         56|
|     4|       101|             NCR|         17|
|     5|       102|             NCR|          3|
|     6|       103| Diebold Nixdorf|        103|
|     7|       104|             NCR|         58|
|     8|       105| Diebold Nixdorf|         76|
|     9|       106|             NCR|         55|
|    10|       107| Diebold Nixdorf|         62|
+------+----------+----------------+-----------+
only showing top 10 rows

In [25]:
df_atm.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- atm_id: integer (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- location_id: integer (nullable = true)

## Creating Date dimension

In [35]:
df_date = df.select("year","month","day","hour","weekday").distinct()
df_date.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+--------+
|year|  month|day|hour| weekday|
+----+-------+---+----+--------+
|2017|January|  5|  21|Thursday|
|2017|January| 22|  15|  Sunday|
|2017|  April|  7|   9|  Friday|
|2017|January| 23|  18|  Monday|
|2017|  March| 17|   1|  Friday|
+----+-------+---+----+--------+
only showing top 5 rows

In [36]:
df_date.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [ ]:
# If using Spark version > 3.0, then use this for proper conversion to timestamp

# spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [41]:
# create the timestamp using year, month, day, hour columns
# date_id is the primary key generated using row_number

window_spec = Window.partitionBy().orderBy('year','month','day','hour')

df_date = df_date.select(row_number().over(window_spec).alias('date_id'),
                         to_timestamp(
                            concat_ws(
                                " ",
                                df["year"].cast("string"),  # Cast year to string
                                df["month"],
                                df["day"].cast("string"),   # Cast day to string
                                df["hour"].cast("string"),  # Cast hour to string
                                    ),
                    "yyyy MMMM dd HH").alias('full_date_time'),  #"yyyy MMMM dd HH" is the format of input
                         '*')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df_date.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+-----+---+----+--------+
|date_id|     full_date_time|year|month|day|hour| weekday|
+-------+-------------------+----+-----+---+----+--------+
|      1|2017-04-01 00:00:00|2017|April|  1|   0|Saturday|
|      2|2017-04-01 01:00:00|2017|April|  1|   1|Saturday|
|      3|2017-04-01 02:00:00|2017|April|  1|   2|Saturday|
|      4|2017-04-01 03:00:00|2017|April|  1|   3|Saturday|
|      5|2017-04-01 04:00:00|2017|April|  1|   4|Saturday|
|      6|2017-04-01 05:00:00|2017|April|  1|   5|Saturday|
|      7|2017-04-01 06:00:00|2017|April|  1|   6|Saturday|
|      8|2017-04-01 07:00:00|2017|April|  1|   7|Saturday|
|      9|2017-04-01 08:00:00|2017|April|  1|   8|Saturday|
|     10|2017-04-01 09:00:00|2017|April|  1|   9|Saturday|
+-------+-------------------+----+-----+---+----+--------+
only showing top 10 rows

In [43]:
df_date1.take(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(date_id=1, full_date_time=datetime.datetime(2017, 4, 1, 0, 0), year=2017, month='April', day=1, hour=0, weekday='Saturday'), Row(date_id=2, full_date_time=datetime.datetime(2017, 4, 1, 1, 0), year=2017, month='April', day=1, hour=1, weekday='Saturday'), Row(date_id=3, full_date_time=datetime.datetime(2017, 4, 1, 2, 0), year=2017, month='April', day=1, hour=2, weekday='Saturday'), Row(date_id=4, full_date_time=datetime.datetime(2017, 4, 1, 3, 0), year=2017, month='April', day=1, hour=3, weekday='Saturday'), Row(date_id=5, full_date_time=datetime.datetime(2017, 4, 1, 4, 0), year=2017, month='April', day=1, hour=4, weekday='Saturday')]

In [44]:
df_date.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_id: integer (nullable = true)
 |-- full_date_time: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- weekday: string (nullable = true)

## Creating Card type dimension

In [45]:
df_card = df.select('card_type').distinct()
df_card.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           card_type|
+--------------------+
|Visa Dankort - on-us|
|  Mastercard - on-us|
|         HÃƒÂ¦vekort|
|            VisaPlus|
|     Dankort - on-us|
+--------------------+
only showing top 5 rows

In [46]:
df_card = df_card.select(row_number().over(Window.partitionBy().orderBy('card_type')).alias('card_type_id'), 'card_type')

df_card.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------------+
|card_type_id|          card_type|
+------------+-------------------+
|           1|             CIRRUS|
|           2|            Dankort|
|           3|    Dankort - on-us|
|           4|        HÃƒÂ¦vekort|
|           5|HÃƒÂ¦vekort - on-us|
+------------+-------------------+
only showing top 5 rows

In [47]:
df_card.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [49]:
df_card.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|           1|              CIRRUS|
|           2|             Dankort|
|           3|     Dankort - on-us|
|           4|         HÃƒÂ¦vekort|
|           5| HÃƒÂ¦vekort - on-us|
|           6|             Maestro|
|           7|          MasterCard|
|           8|  Mastercard - on-us|
|           9|                VISA|
|          10|        Visa Dankort|
|          11|Visa Dankort - on-us|
|          12|            VisaPlus|
+------------+--------------------+

In [48]:
df_card.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- card_type_id: integer (nullable = true)
 |-- card_type: string (nullable = true)

## Creating fact table

In [82]:
# Creating all the views for all dimensions and fact table

df_location.createOrReplaceTempView('loc')
df_atm.createOrReplaceTempView('atm')
df_date.createOrReplaceTempView('date')
df_card.createOrReplaceTempView('card')
df.createOrReplaceTempView('fact')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
# joining with card dimension

df_fact = spark.sql("select card.card_type_id, fact.* \
                    from fact left join card on fact.card_type = card.card_type")
df_fact.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----+-------+---+-------+----+----------+------+----------------+------------+-----------------+-----------------+-----------+-------+-------+--------+---------------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+-------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+
|card_type_id|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|   atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency|      card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|   temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+------------+----+-------+---+-------+----+----------+------+----------------+------------+-----------------+-----------------+-----------+-------+-------+--------+---------------+---------

In [84]:
# create a view for the fact table based on updated columns after previous join

df_fact.createOrReplaceTempView('fact')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
# join with date dimension

df_fact = spark.sql("select date.date_id, fact.* from fact left join date on \
                    fact.year = date.year and \
                    fact.month = date.month and \
                    fact.day = date.day and \
                    fact.hour = date.hour and \
                    fact.weekday = date.weekday ")

df_fact.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+----+-------+---+-------+----+----------+------+----------------+------------+-----------------+-----------------+-----------+-------+-------+--------+---------------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+-------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+
|date_id|card_type_id|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|   atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency|      card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|   temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+-------+------------+----+-------+---+-------+----+----------+------+----------------+------------+-----------------+-----------------+-----------+-------+-------+--------+-

In [86]:
df_fact.createOrReplaceTempView('fact')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
# join with location dimension

df_fact = spark.sql("select loc.location_id weather_loc_id, fact.* \
                     from fact left join loc on \
                     fact.atm_location = loc.atm_location and \
                     fact.atm_streetname = loc.atm_streetname and \
                     fact.atm_street_number = loc.atm_street_number and \
                     fact.atm_zipcode = loc.atm_zipcode and \
                     fact.atm_lat = loc.atm_lat and \
                     fact.atm_lon = loc.atm_lon ")

df_fact.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+------------+----+-------+---+-------+----+----------+------+----------------+------------+----------------+-----------------+-----------+-------+-------+--------+---------------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+-------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+
|weather_loc_id|date_id|card_type_id|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|  atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency|      card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|   temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------------+-------+------------+----+-------+---+-------+----+----------+------+----------------+------------+----------------+--------------

In [88]:
df_fact.createOrReplaceTempView('fact')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
# join with atm dimension
# note that atm_id of fact will be mapped to atm_number of atm dimension

df_fact = spark.sql("select atm.atm_id atm_id_dim, fact.* \
                    from fact left join atm on \
                    fact.atm_id = atm.atm_number and \
                    fact.atm_manufacturer = atm.atm_manufacturer and \
                    fact.weather_loc_id = atm.location_id ")

df_fact.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------+-------+------------+----+-------+---+-------+----+----------+------+----------------+--------------+-----------------+-----------------+-----------+-------+-------+--------+---------------+------------------+----------+------------+--------------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+
|atm_id_dim|weather_loc_id|date_id|card_type_id|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|  atm_location|   atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency|      card_type|transaction_amount|   service|message_code|        message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+----------+--------------+-------+------------+----+-------+---+-------+----+----------+------+--------

In [90]:
df_fact.createOrReplaceTempView('fact')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
# selecting only the relevant columns in fact table

df_fact = df_fact.select(row_number().over(Window.partitionBy().orderBy('date_id','atm_id_dim','weather_loc_id','card_type_id')).alias('trans_id'),
                         col('atm_id_dim').alias('atm_id'),
                         'weather_loc_id','date_id', 'card_type_id', 'atm_status', 'currency','service', 'transaction_amount',
                         'message_code','message_text','rain_3h','clouds_all','weather_id','weather_main', 'weather_description'
                         )

df_fact.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|       1|     1|            74|      1|           7|    Active|     DKK|Withdrawal|              2033|        null|        null|    0.0|        56|       701|        Mist|               mist|
|       2|     1|            74|      1|           8|    Active|     DKK|Withdrawal|              4502|        null|        null|    0.0|        56|       701|        Mist|               mist|
|       3|     1|            74|   

In [92]:
df_fact.createOrReplaceTempView('fact')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
df_fact.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [94]:
df_fact.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trans_id: integer (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- weather_loc_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- service: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- rain_3h: double (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)

In [95]:
null_counts = df_fact.select([sum(col(column).isNull().cast("int")).alias(column) for column in df_fact.columns])
null_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|       0|     0|             0|      0|           0|         0|       0|      0|                 0|     2459009|     2459010|   8087|      8087|      8087|        8087|               8087|
+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+------------+-------------------+

In [106]:
df_fact.filter(col('rain_3h').isNull()).select("clouds_all","message_text").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+
|clouds_all|message_text         |
+----------+---------------------+
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
|null      |Suspected malfunction|
+----------+---------------------+
only showing top 20 rows

In [96]:
df_fact.select('message_text').distinct().show(10, truncate=False)

# message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------+
|message_text                                                                              |
+------------------------------------------------------------------------------------------+
|Suspected malfunction|0.000|56.643|10|2620167|0.000|278|1038|88|8|285.000|0|44|802|Clouds |
|Suspected malfunction|0.000|55.490|9|2618528|0.000|280|1020|100|3|150.000|0|92|300|Drizzle|
|Timed-out taking card|0.000|56.715|10|2620952|0.000|276|998|88|7|192.000|0|92|500|Rain    |
|Suspected malfunction|0.000|55.060|11|2612045|0.000|288|1021|87|3|80.000|0|40|802|Clouds  |
|Suspected malfunction|0.000|56.567|9|2613731|0.000|279|1033|100|4|250.000|0|88|500|Rain   |
|Suspected malfunction|0.000|57.048|10|2616235|0.000|289|1010|68|9|39.000|0|68|803|Clouds  |
|Suspected malfunction|0.000|57.441|11|2621927|0.000|285|1030|100|8|120.000|0|92|500|Rain  |
|Suspected malfunction|0.000|55.506|10|2616933|0.000|293|1017|68|3|150

In [ ]:
df_fact.select('message_text', split(col("message_text"), "\\|")[-4].alias('val')).show(10)

In [97]:
df_fact = df_fact.withColumn(
    "rain_3h",
    when(col("message_text").contains("|"), split(col("message_text"), "\\|")[-4])
    .otherwise(col("rain_3h"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
df_fact = df_fact.withColumn(
    "clouds_all",
    when(col("message_text").contains("|"), split(col("message_text"), "\\|")[-3])
    .otherwise(col("clouds_all"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
df_fact = df_fact.withColumn(
    "weather_id",
    when(col("message_text").contains("|"), split(col("message_text"), "\\|")[-2])
    .otherwise(col("weather_id"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
df_fact = df_fact.withColumn(
    "weather_main",
    when(col("message_text").contains("|"), split(col("message_text"), "\\|")[-1])
    .otherwise(col("weather_main"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
df_fact = df_fact.withColumn(
    "message_text",
    when(col("message_text").contains("|"), split(col("message_text"), "\\|")[0])
    .otherwise(col("message_text"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
df_fact.select('message_text').distinct().show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+
|message_text                  |
+------------------------------+
|Suspected malfunction         |
|null                          |
|No response received from host|
|Timed-out taking card         |
|Timed-out taking money        |
+------------------------------+

In [103]:
df_fact.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+------------------+----------------+-----------------+-----------------+----------+--------+----------+------------------+-----------------+--------------------+-------------------+-----------------+------------------+------------+--------------------+
|summary|         trans_id|            atm_id|  weather_loc_id|          date_id|     card_type_id|atm_status|currency|   service|transaction_amount|     message_code|        message_text|            rain_3h|       clouds_all|        weather_id|weather_main| weather_description|
+-------+-----------------+------------------+----------------+-----------------+-----------------+----------+--------+----------+------------------+-----------------+--------------------+-------------------+-----------------+------------------+------------+--------------------+
|  count|          2468572|           2468572|         2468572|          2468572|          2468572|   2468572| 2468572|   2468572|           2468572|           

In [104]:
null_counts = df_fact.select([sum(col(column).isNull().cast("int")).alias(column) for column in df_fact.columns])
null_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|       0|     0|             0|      0|           0|         0|       0|      0|                 0|     2459009|     2459010|   8087|      8087|      8087|        8087|               8087|
+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+------------+-------------------+

## Writing tables in S3

In [ ]:
dim_atm_path = "s3://atm-data-model/DIM_ATM/" 

# Write atm dimension to S3 in CSV format
df_atm.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(dim_atm_path)

In [ ]:
dim_date_path = "s3://atm-data-model/DIM_DATE/" 

# Write date dimension to S3 in CSV format
df_date.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(dim_date_path)

In [ ]:
dim_card_path = "s3://atm-data-model/DIM_CARD_TYPE/" 

# Write card dimension to S3 in CSV format
df_card.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(dim_card_path)

In [ ]:
dim_location_path = "s3://atm-data-model/DIM_LOCATION/" 

# Write location dimension to S3 in CSV format
df_location.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(dim_location_path)

In [ ]:
fact_path = "s3://atm-data-model/FACT_ATM_TRANS/" 

# Write fact table to S3 in CSV format
df_fact.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(fact_path)